In [1]:
import pandas as pd
import pickle
from ast import literal_eval
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, cohen_kappa_score, f1_score , confusion_matrix
from nltk.corpus import stopwords
from sklearn.svm import LinearSVC , NuSVC
from germeval import *
from utils import subtask_A_evaluation ,subtask_A_confusion_matrix , subtask_A_classification_report

In [2]:
from bpemb import BPEmb
bpemb_de = BPEmb(lang="de", vs=25000, dim=300)

def bpemb_tokenize(sentence):
    return [i.replace("▁","") for i in bpemb_de.encode(sentence)]


In [3]:
mlb =  pickle.load(open("tl_mlb_author_year_without_num.pickle",'rb'))
year_lb = pickle.load(open("tl_lb_author_year_without_num.pickle",'rb'))
vectorizer =  pickle.load(open("tl_train_author_vectorizer_without_num.pickle",'rb'))

X_train = pickle.load(open("tl_train_author_X_train_without_num.pickle", "rb"))
X_test = pickle.load(open("tl_train_author_X_test_without_num.pickle", "rb"))
y_train = pickle.load( open("tl_train_author_y_train_without_num.pickle", "rb"))
y_test = pickle.load(open("tl_train_author_y_test_without_num.pickle", "rb"))
le_mapping = pickle.load(open("tl_train_author_le_mapping.pickle", "rb")) # LabelEncoder Mapping

In [4]:
def build_model(clf, X_train,y_train):
    clf.fit(X_train, y_train)
    return clf

## Linear SVC

In [5]:
X_train.shape

(16929, 557737)

In [6]:
X_test.shape

(4233, 557737)

In [7]:
%%time
X_full = np.concatenate((X_train.toarray(),X_test.toarray()))
Y_full = np.concatenate((y_train,y_test))

CPU times: user 21.2 s, sys: 59.3 s, total: 1min 20s
Wall time: 1min 20s


In [8]:
X_full.shape

(21162, 557737)

In [9]:
y_train.value_counts()

Literatur & Unterhaltung      7305
Kinderbuch & Jugendbuch       3142
Sachbuch                      2646
Ratgeber                      2017
Ganzheitliches Bewusstsein     848
Glaube & Ethik                 687
Architektur & Garten           165
Künste                         119
Name: toplevel, dtype: int64

In [10]:
del X_test
del X_train
del y_train
del y_test
import gc
gc.collect()

98

In [11]:
# from sklearn.grid_search import GridSearchCV

# param_grid = {'C':[1,10,100,1000],'max_iter' : [1000,2000]}
# grid = GridSearchCV(LinearSVC(),param_grid,refit = True, verbose=2)
# grid.fit(X_train.toarray(),y_train)
Y_full

array(['Ratgeber', 'Glaube & Ethik', 'Sachbuch', ..., 'Ratgeber',
       'Kinderbuch & Jugendbuch', 'Literatur & Unterhaltung'],
      dtype=object)

In [12]:
# del X_full
# del Y_full
# gc.collect()
# weights = {'Kinderbuch & Jugendbuch': 1.8, 'Ratgeber': 3, 'Sachbuch': 2,'Glaube & Ethik' : 2 ,'Künste' : 6,'Architektur & Garten' : 6 }
weights = {'Kinderbuch & Jugendbuch': 1.8, 'Ratgeber': 3, 'Sachbuch': 2,'Glaube & Ethik' : 2 ,'Künste' : 20,'Architektur & Garten' : 20 }

In [13]:
%%time
# data = X_train.toarray(), X_test.toarray(), y_train, y_test
lsvcclf = LinearSVC(random_state=42,max_iter=3000,verbose=1,class_weight= weights)
clf = build_model(lsvcclf,X_full,Y_full)

[LibLinear]CPU times: user 1min 7s, sys: 3.04 s, total: 1min 10s
Wall time: 1min 10s


In [14]:
lsvcclf.get_params()

{'C': 1.0,
 'class_weight': {'Architektur & Garten': 20,
  'Glaube & Ethik': 2,
  'Kinderbuch & Jugendbuch': 1.8,
  'Künste': 20,
  'Ratgeber': 3,
  'Sachbuch': 2},
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 3000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': 42,
 'tol': 0.0001,
 'verbose': 1}

### Run the predictions on valiation for submission

In [15]:
validation_data = pd.read_csv("../../data/labeled_validation_data.csv")

In [16]:
def combine_title_desc(row):
    """Combines the Title and Description fields in the given row
    and returns the combined result"""
    return str(row["title"]) + " " + str(row["description"])
    #return str(row["title"]) + " " + str(row["description"]) + " " + str(row["author"])

In [17]:
validation_tf_idf = vectorizer.transform(validation_data.apply(lambda row: combine_title_desc(row),axis=1))

/home/evenuma/anaconda3/envs/deeplearning_36/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['be', 'chem', 'ches', 'd', 'e', 'em', 'en', 'eur', 'ig', 'igem', 'iges', 'jen', 'm', 'r', 'sel', 'selben', 'sol', 'st', 'uer', 'ure', 'uren'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [18]:
validation_data["authors"] = validation_data["author"].apply(lambda x:[ i.strip() for i in str(x).split(",")])

In [19]:
validation_data["published_date_parsed"] = pd.to_datetime(validation_data["published_date"],infer_datetime_format=True)

In [20]:
validation_data["year"] = validation_data["published_date_parsed"].apply(lambda x: x.year)

In [21]:
validation_tf_idf.shape

(2079, 549048)

In [22]:
validation_data["authors"].values

array([list(['Hans-Martin Lübking']), list(['Cate Tiernan']),
       list(['Titus Müller']), ..., list(['Arthur C. Clarke']),
       list(['Paul Levine']), list(['Robert Greene'])], dtype=object)

In [23]:
len(validation_data)

2079

In [24]:
pickle.dump(validation_data["authors"], open("validation_authors", "wb"))

In [25]:
from scipy import sparse

validation_author = mlb.transform(validation_data["authors"].values)
validation_year = year_lb.transform(validation_data["year"].values)

validation_all = sparse.hstack((validation_tf_idf,
                    sparse.csr_matrix(validation_author)
                    ,sparse.csr_matrix(validation_year)))

In [26]:
validation_all

<2079x557737 sparse matrix of type '<class 'numpy.float64'>'
	with 291449 stored elements in COOrdinate format>

In [27]:
predictions = clf._predict_proba_lr(validation_all)

In [28]:
del X_full
del Y_full

In [29]:
import gc
gc.collect()

14

In [30]:
class_counts = pd.read_csv("class_count_predictions_from_svc_validation_set.csv",header=None)

In [31]:
def getClasses(class_probs,class_count):
    arr = np.array(class_probs)
    return arr.argsort()[-1 * class_count:][::-1]

In [32]:
y_prob_list = [i.tolist() for i in predictions]

In [33]:
validation_data["class_probs"] = y_prob_list

In [34]:
validation_data["class_count"] = class_counts.values

In [35]:
# validation_data["predictions"]  = 
validation_data["class_predictions_e"] = validation_data.apply(lambda x:getClasses(x["class_probs"],x["class_count"]),axis=1)

In [36]:
le_mapping_inv = {v: k for k, v in le_mapping.items()}

def reverseMap(number):
    return le_mapping_inv[number]

In [37]:
validation_data["class_predictions"] = validation_data["class_predictions_e"].apply(lambda x: [reverseMap(i) for i in x])

In [38]:
def formatOutput(isbn,class_predictions):
    classes = ["\t"+str(i) for i in class_predictions]
    return str(isbn) + "".join(classes) + "\n"

In [39]:
validation_data["class_predictions_top_level"] = validation_data["class_predictions"].apply(lambda cat_paths: [x.split(" > ")[0] for x in cat_paths])

In [40]:
validation_data["categories"] = validation_data["categories"].apply(lambda x : literal_eval(x))

In [41]:
validation_data["actual_toplevel"] = validation_data["categories"].apply(lambda x:list(set([i.split(" > ")[0] for i in x]))).values

In [42]:
validation_data.head(2)

,Unnamed: 0,title,description,categories,author,published_date,isbn,authors,published_date_parsed,year,class_probs,class_count,class_predictions_e,class_predictions,class_predictions_top_level,actual_toplevel
0,14548,Selbstständig werden im Glauben,Die Konfirmandenzeit wird für Jugendliche beso...,[Glaube & Ethik > Gemeindearbeit > Konfirmatio...,Hans-Martin Lübking,2013-07-22,9783579074030,[Hans-Martin Lübking],2013-07-22,2013,"[0.09751506993501706, 0.09792159042868111, 0.3...",1,[2],[Glaube & Ethik],[Glaube & Ethik],[Glaube & Ethik]
1,14549,Das Buch der Schatten - Schwarze Seelen,Als Morgan von einer prophetischen Vision heim...,"[Kinderbuch & Jugendbuch > Echtes Leben, Reali...",Cate Tiernan,2013-06-10,9783570380093,[Cate Tiernan],2013-06-10,2013,"[0.10211832175306117, 0.10370471720071815, 0.0...",1,[3],[Kinderbuch & Jugendbuch],[Kinderbuch & Jugendbuch],[Kinderbuch & Jugendbuch]


In [43]:
y_true = validation_data["actual_toplevel"].values
y_pred = validation_data["class_predictions_top_level"]
subtask_A_evaluation(y_true,y_pred)

[0.8228699551569507,
 0.8826358826358827,
 0.8517057321884429,
 0.8177008177008177]

In [44]:
validation_data[["isbn","class_predictions_top_level"]].to_csv("top_level_predictions_svm.csv",index=False)

In [45]:
max(validation_data.class_count.tolist())

1

In [46]:
y_true = validation_data["actual_toplevel"].values
y_pred = validation_data["class_predictions_top_level"]
subtask_A_confusion_matrix(y_true,y_pred)

array([[[2062,    0],
        [   5,   12]],

       [[1963,    4],
        [  40,   72]],

       [[1984,    5],
        [  32,   58]],

       [[1779,   12],
        [  76,  212]],

       [[2061,    0],
        [  10,    8]],

       [[ 844,  128],
        [  58, 1049]],

       [[1779,   40],
        [  70,  190]],

       [[1686,   55],
        [ 104,  234]]])

In [47]:
y_true = validation_data["actual_toplevel"].values
y_pred = validation_data["class_predictions_top_level"]
print(subtask_A_classification_report(y_true,y_pred))

                            precision    recall  f1-score   support

      Architektur & Garten       1.00      0.71      0.83        17
Ganzheitliches Bewusstsein       0.95      0.64      0.77       112
            Glaube & Ethik       0.92      0.64      0.76        90
   Kinderbuch & Jugendbuch       0.95      0.74      0.83       288
                    Künste       1.00      0.44      0.62        18
  Literatur & Unterhaltung       0.89      0.95      0.92      1107
                  Ratgeber       0.83      0.73      0.78       260
                  Sachbuch       0.81      0.69      0.75       338

                 micro avg       0.88      0.82      0.85      2230
                 macro avg       0.92      0.69      0.78      2230
              weighted avg       0.88      0.82      0.85      2230
               samples avg       0.88      0.85      0.86      2230



In [48]:
y_true

array([list(['Glaube & Ethik']), list(['Kinderbuch & Jugendbuch']),
       list(['Literatur & Unterhaltung']), ...,
       list(['Literatur & Unterhaltung']),
       list(['Literatur & Unterhaltung']),
       list(['Sachbuch', 'Ganzheitliches Bewusstsein', 'Ratgeber'])],
      dtype=object)

In [49]:
submissions = validation_data.apply(lambda x:formatOutput(x["isbn"],x["class_predictions_top_level"]),axis=1).values

In [50]:
submissions = np.insert(submissions,0,"subtask_a\n")

In [51]:
submission_format = "".join(submissions)

In [52]:
submission_file = open("../../data/answers_svm_with_class_unsampling_with_author_year_test_set.txt",'w')

In [53]:
submission_file.write(submission_format)

71893

## ELI5 Analysis of results

In [52]:
import eli5
eli5.explain_weights(lsvcclf)

Explanation(estimator="LinearSVC(C=1.0,\n          class_weight={'Architektur & Garten': 6, 'Glaube & Ethik': 2,\n                        'Kinderbuch & Jugendbuch': 1.8, 'Künste': 6,\n                        'Ratgeber': 3, 'Sachbuch': 2},\n          dual=True, fit_intercept=True, intercept_scaling=1,\n          loss='squared_hinge', max_iter=3000, multi_class='ovr', penalty='l2',\n          random_state=42, tol=0.0001, verbose=1)", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='Architektur & Garten', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='x189130', weight=1.8986682894870917, std=None, value=None), FeatureWeight(feature='x551287', weight=1.230031583440756, std=None, value=None), FeatureWeight(feature='x76318', weight=1.22880462181355, std=None, value=None), FeatureWeight(feature='x58575', weight=1.2242454917703847, std=None, value=None), FeatureWeight(feature='x553184', weight=1.2046613469926633, std=None, value=None), FeatureWeight(feature='x217579', weight=1.1807730305344486, std=None, value=None), FeatureWeight(feature='x556228', weight=1.128707299701547, std=None, value=None), FeatureWeight(feature='x550207', weight=1.120190217768477, std=None, value=None), FeatureWeight(feature='x556941', weight=1.112877460417924, std=None, value=None), FeatureWeight(feature='x556883', weight=1.093467909946022, std=None, value=None), FeatureWeight(feature='x557582', weight=1.0908459190141733, std=None, value=None), FeatureWeight(feature='x549089', weight=1.0853319606027607, std=None, value=None), FeatureWeight(feature='x512877', weight=1.0399007234541309, std=None, value=None), FeatureWeight(feature='x549673', weight=0.9940479207382191, std=None, value=None), FeatureWeight(feature='x551081', weight=0.9867823348928054, std=None, value=None), FeatureWeight(feature='x552344', weight=0.9831000630218878, std=None, value=None), FeatureWeight(feature='x549982', weight=0.9682991479673254, std=None, value=None), FeatureWeight(feature='x553365', weight=0.9226669850674148, std=None, value=None), FeatureWeight(feature='x549189', weight=0.9129188161487145, std=None, value=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-0.9689336713776673, std=None, value=None)], pos_remaining=41469, neg_remaining=225795), proba=None, score=None, weighted_spans=None), TargetExplanation(target='Ganzheitliches Bewusstsein', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='x223680', weight=1.5277153984591876, std=None, value=None), FeatureWeight(feature='x223573', weight=1.4604407749599548, std=None, value=None), FeatureWeight(feature='x555133', weight=1.4240337013077782, std=None, value=None), FeatureWeight(feature='x551293', weight=1.28484978067581, std=None, value=None), FeatureWeight(feature='x552397', weight=1.18554795030805, std=None, value=None), FeatureWeight(feature='x421300', weight=1.169048929652411, std=None, value=None), FeatureWeight(feature='x556099', weight=1.1676321901859943, std=None, value=None), FeatureWeight(feature='x334874', weight=1.165736516086417, std=None, value=None), FeatureWeight(feature='x553912', weight=1.124291013552691, std=None, value=None), FeatureWeight(feature='x223598', weight=1.0645229926985795, std=None, value=None), FeatureWeight(feature='x550763', weight=1.0526078543973643, std=None, value=None), FeatureWeight(feature='x555011', weight=1.0417451499423565, std=None, value=None), FeatureWeight(feature='x552814', weight=1.0174413366238075, std=Non

In [58]:
dir(eli5)
# dir(eli5)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_decision_path',
 '_feature_importances',
 '_feature_names',
 '_feature_weights',
 '_graphviz',
 'absolute_import',
 'base',
 'base_utils',
 'explain',
 'explain_prediction',
 'explain_prediction_df',
 'explain_prediction_dfs',
 'explain_prediction_lightgbm',
 'explain_prediction_sklearn',
 'explain_weights',
 'explain_weights_df',
 'explain_weights_dfs',
 'explain_weights_lightgbm',
 'explain_weights_sklearn',
 'format_as_dataframe',
 'format_as_dataframes',
 'format_as_dict',
 'format_as_html',
 'format_as_text',
 'format_html_styles',
 'formatters',
 'ipython',
 'lightgbm',
 'permutation_importance',
 'show_prediction',
 'show_weights',
 'sklearn',
 'transform',
 'transform_feature_names',
 'utils']

In [93]:
eli5.explain_prediction(lsvcclf,validation_csr[2021,])

Explanation(estimator="LinearSVC(C=1.0,\n          class_weight={'Architektur & Garten': 6, 'Glaube & Ethik': 2,\n                        'Kinderbuch & Jugendbuch': 1.8, 'Künste': 6,\n                        'Ratgeber': 3, 'Sachbuch': 2},\n          dual=True, fit_intercept=True, intercept_scaling=1,\n          loss='squared_hinge', max_iter=3000, multi_class='ovr', penalty='l2',\n          random_state=42, tol=0.0001, verbose=1)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='Architektur & Garten', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='x42950', weight=0.010019777598699552, std=None, value=0.052587367926256186), FeatureWeight(feature='x355097', weight=0.008433018265659025, std=None, value=0.04633542420611241), FeatureWeight(feature='x245972', weight=0.007826668875058937, std=None, value=0.07241682427194142), FeatureWeight(feature='x403897', weight=0.007082438321258365, std=None, value=0.05304646858828182), FeatureWeight(feature='x380246', weight=0.0050270705236802144, std=None, value=0.06364655648411828), FeatureWeight(feature='x271438', weight=0.003240781518494097, std=None, value=0.06779771180478215), FeatureWeight(feature='x120959', weight=0.002945670496939076, std=None, value=0.06135377568559595), FeatureWeight(feature='x269348', weight=0.0025137165318639266, std=None, value=0.08122741872138911), FeatureWeight(feature='x440261', weight=0.0021158320056201855, std=None, value=0.07781215934101277), FeatureWeight(feature='x9116', weight=0.0016449765496164472, std=None, value=0.06440146870274069), FeatureWeight(feature='x557733', weight=0.0014101702482304855, std=None, value=1.0), FeatureWeight(feature='x440259', weight=0.0013861475728421254, std=None, value=0.045936959224501715), FeatureWeight(feature='x351177', weight=0.0010694107405448373, std=None, value=0.07139364477231709), FeatureWeight(feature='x503672', weight=0.0007050004893055337, std=None, value=0.05940699430724982), FeatureWeight(feature='x388124', weight=0.0006326720578512698, std=None, value=0.040056657072557664), FeatureWeight(feature='x120961', weight=0.0003564581620782854, std=None, value=0.08609925559249768), FeatureWeight(feature='x379538', weight=0.00024108486121056144, std=None, value=0.058187357898374904), FeatureWeight(feature='x8261', weight=3.740840762875874e-20, std=None, value=0.11041013147723464)], neg=[FeatureWeight(feature='<BIAS>', weight=-0.9689336713776673, std=None, value=1.0), FeatureWeight(feature='x551971', weight=-0.02526564521389673, std=None, value=1.0), FeatureWeight(feature='x16444', weight=-0.014755008670337383, std=None, value=0.06519803058079474), FeatureWeight(feature='x503665', weight=-0.009262382796971077, std=None, value=0.0660856602100129), FeatureWeight(feature='x23656', weight=-0.005781581227160693, std=None, value=0.026501398171218472), FeatureWeight(feature='x138756', weight=-0.005739351015386523, std=None, value=0.04443812438099546), FeatureWeight(feature='x479623', weight=-0.005469434451942609, std=None, value=0.045819775655164895), FeatureWeight(feature='x307650', weight=-0.005455754131486998, std=None, value=0.028389052643141573), FeatureWeight(feature='x4293', weight=-0.0044312727772122884, std=None, value=0.11773834737305452), FeatureWeight(feature='x55729', weight=-0.003932354326403666, std=None, value=0.09049346289321925), FeatureWeight(feature='x73605', weight=-0.003501313881347277, std=None, value=0.05066955275098643), FeatureWeight(feature='x56105', weight=-0.00344484515552642, std=None, value=0.23706411961723436), FeatureWeight(feature='x278903', weight=-0.0033336414250795523, std=None, value=0.04721444377663392), FeatureWeight(feature='x280658', weight=-0.0032049444293226416, std=None, value=0.15934243355273287), FeatureWeight(feature='x109675', weight=-0.0030253677877833253, std=None, value=0.053505197234210676), FeatureWeight(feature='x293177', weight=-0.003016122002105476, std=None, value=0.07288139515911392), FeatureWeight(f

In [86]:
pd.set_option('display.max_colwidth', -1)

In [95]:
# import eli5
# from eli5.lime import TextExplainer

# te = TextExplainer(random_state=42)
# te.fit(validation_csr[2021,], lsvcclf._predict_proba_lr)
# te.show_prediction(target_names=le_mapping.keys())

In [238]:
validation_data[validation_data["actual_toplevel"].apply(lambda x: 'Literatur & Unterhaltung' in x)][["class_probs","class_predictions","actual_toplevel","class_count"]]

,class_probs,class_predictions,actual_toplevel,class_count
2,"[0.10176139047229471, 0.10034340459531735, 0.11899230127719931, 0.10083888902527291, 0.09877020308626638, 0.3011165363411506, 0.09302863035796799, 0.0851486448445308]",[Literatur & Unterhaltung],[Literatur & Unterhaltung],1
4,"[0.09947411214055941, 0.1006119545634914, 0.0923209309685662, 0.15006827885317528, 0.09652199517300623, 0.2840845695015635, 0.0973861587902958, 0.07953200000934217]",[Literatur & Unterhaltung],[Literatur & Unterhaltung],1
5,"[0.09996371035975586, 0.1106844950149916, 0.10459593733015032, 0.09692840089077716, 0.08916959397974776, 0.3198172036102407, 0.08457679177137276, 0.09426386704296395]",[Literatur & Unterhaltung],[Literatur & Unterhaltung],1
6,"[0.10323601369757439, 0.09317998562485248, 0.09747528122418334, 0.09171074950832725, 0.10330877469997381, 0.22160307325039083, 0.09867801858414632, 0.19080810341055152]",[Literatur & Unterhaltung],[Literatur & Unterhaltung],1
7,"[0.10658772857039993, 0.10485301150159572, 0.10645535954993907, 0.1500596736092641, 0.10249957803025704, 0.21776862814846806, 0.11002358299575948, 0.10175243759431665]",[Literatur & Unterhaltung],[Literatur & Unterhaltung],1
8,"[0.09799650998229127, 0.11147832188999675, 0.09692099463720068, 0.09183477790644891, 0.09852775601664103, 0.30462124799735746, 0.09951388498268993, 0.0991065065873739]",[Literatur & Unterhaltung],[Literatur & Unterhaltung],1
10,"[0.10917319929551024, 0.09500564783193124, 0.09339317813200317, 0.08933976544749977, 0.10740374628336767, 0.33396287362901667, 0.08548279798534676, 0.08623879139532446]",[Literatur & Unterhaltung],[Literatur & Unterhaltung],1
11,"[0.10601157903302762, 0.10609657250427433, 0.09893003781532793, 0.12653785707612988, 0.09828008523691371, 0.2288076456026211, 0.11123063922882229, 0.12410558350288313]",[Literatur & Unterhaltung],[Literatur & Unterhaltung],1
13,"[0.10490561757179506, 0.0932787635722231, 0.09453694002005113, 0.10748784483440875, 0.10100484228864909, 0.3058273969987852, 0.09126109157540943, 0.10169750313867815]",[Literatur & Unterhaltung],[Literatur & Unterhaltung],1
14,"[0.10080756154223681, 0.10805454098054262, 0.09586374170860981, 0.10861362836014954, 0.09751476646459303, 0.24122380733022245, 0.08943013513687055, 0.1584918184767751]",[Literatur & Unterhaltung],[Literatur & Unterhaltung],1


In [91]:
le_mapping

{'Architektur & Garten': 0,
 'Ganzheitliches Bewusstsein': 1,
 'Glaube & Ethik': 2,
 'Kinderbuch & Jugendbuch': 3,
 'Künste': 4,
 'Literatur & Unterhaltung': 5,
 'Ratgeber': 6,
 'Sachbuch': 7}

In [239]:
mis_match = validation_data[validation_data["actual_toplevel"].apply(lambda x: 'Literatur & Unterhaltung' in x)]

In [240]:
mis_match["actual_count"] = mis_match["actual_toplevel"].apply(lambda x:len(x))

/home/evenuma/anaconda3/envs/deeplearning_36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [241]:
mis_match[mis_match["actual_count"] != mis_match["class_count"]]

,Unnamed: 0,title,description,categories,author,published_date,isbn,authors,published_date_parsed,year,class_probs,class_count,class_predictions_e,class_predictions,class_predictions_top_level,actual_toplevel,actual_count
15,14563,Ohne Liebe trauern die Sterne,"Sie ist eine der beliebtesten deutschen Schauspielerinnen: Hannelore Hoger. Als eigenwillige Kommissarin Bella Block wurde sie populär, doch das Spektrum ihrer zahlreichen Rollen in Film- und & Theaterproduktionen ist groß. Offen wie noch nie spricht sie über persönliche Vorlieben und Obsessionen, und erzählt ausführlich aus ihrem Leben: von Kindheit und Jugend in Hamburg, von ihren Anfängen und ersten Erfolgen als Schauspielerin, von der Zusammenarbeit mit Regisseuren wie Peter Zadek, Edgar Reitz und Alexander Kluge, die sie prägten, von Kollegen wie Ulrich Wildgruber, Götz George, Harald Juhnke – und ihrer Tochter Nina. „Hannelore Hoger ist eine unglaublich gute Schauspielerin, die einen Regisseur permanent inspiriert. Sie ist bis in die Knochen eine Komödiantin.” Edgar Reitz","[Sachbuch > Biographien & Autobiographien > Künstler, Dichter, Denker, Sachbuch > Biographien & Autobiographien > Schauspieler, Regisseure, Sachbuch > Lifestyle, Literatur & Unterhaltung > Romane & Erzählungen, Sachbuch > Kunst & Kultur, Sachbuch > Kabarett & Satire]",Hannelore Hoger,2017-06-13,9783837140118,[Hannelore Hoger],2017-06-13,2017,"[0.10180959282982897, 0.10353082026204626, 0.09376280257547408, 0.12645269939035333, 0.1081999598020381, 0.17262388006777646, 0.10789764643994189, 0.1857225986325409]",1,[7],[Sachbuch],[Sachbuch],"[Literatur & Unterhaltung, Sachbuch]",2
20,14568,Märchenwelt,"Märchen sind der älteste Wissensspeicher der Menschheit und ein Spiegel ihrer Träume, Ängste und Wünsche. Michael Köhlmeier, gefeierter Romancier und begnadeter Erzähler, hat einhundert Geschichten aus aller Welt gesammelt und nach Urszenen geordnet. Neben altbekannten Klassikern – etwa den Märchen der Brüder Grimm, von Hans Christian Andersen oder den Geschichten aus Tausendundeiner Nacht – gibt es neue Perlen zu entdecken: isländische Sagen und Märchen aus der Karibik ebenso wie Überlieferungen aus den Steppen Nordamerikas. Ein Schatz für jeden Märchenliebhaber, egal ob groß oder klein.","[Literatur & Unterhaltung > Klassiker & Lyrik > Märchen, Legenden und Sagen, Kinderbuch & Jugendbuch > Echtes Leben, Realistischer Roman]",Michael Köhlmeier,2017-10-09,9783328102212,[Michael Köhlmeier],2017-10-09,2017,"[0.10289048429964959, 0.11815669821642352, 0.09619184946173062, 0.22993903886350903, 0.1028787124788307, 0.15022543007948355, 0.10996945031748397, 0.08974833628288893]",1,[3],[Kinderbuch & Jugendbuch],[Kinderbuch & Jugendbuch],"[Kinderbuch & Jugendbuch, Literatur & Unterhaltung]",2
140,14688,Der Prinz und der Bettelknabe,"Tom ist ein Betteljunge, der sich nichts sehnlicher wünscht, als einmal in seinem Leben ein richtiger Prinz zu sein. Edward hingegen ist ein Prinz, der sich im Schloss langweilt. Durch Zufall treffen sich die beiden Jungen, die sich zum Verwechseln ähnlich sehen. Heimlich tauschen sie ihre Kleider – und eine spannende Geschichte beginnt ... Es gibt Dinge, deren Entdeckung immer wieder aufs Neue Spaß macht: Ein treffliches Beispiel ist diese liebevolle Hörspiel-Inszenierung von Otto Kurth aus dem Jahr 1968. Hans Paetsch, der bekannteste Märchenonkel unserer Kindheit, führt so warmherzig durch diese Verwechslungsgeschichte, dass wir uns für kurze Zeit nicht nur im London der Renaissance-Zeit, sondern im Hörspielhimmel glauben.","[Kinderbuch & Jugendbuch > Abenteuer, Literatur & Unterhaltung > Klassiker & Lyrik, Kinderbuch & Jugendbuch > Liebe, Beziehung und Freundschaft, Kinderbuch & Jugendbuch > Echtes Leben, Realistischer Roman]",Mark Twain,2013-10-22,9783844514605,[Mark Twain],2013-10-22,2013,"[0.09816415228839201, 0.09600991941522911, 0.09907226305150386, 0.18703834494589563, 0.09382253487835414, 0.23030474582426008, 0.08988723598102849, 0.1057008036153365]",1,[5],[Literatur & Unte

## Lets check the confusion matrix (Do not run this , this will take long time and will not yield good results)

In [9]:
y_test.values

array(['Literatur & Unterhaltung', 'Literatur & Unterhaltung', 'Sachbuch',
       ..., 'Ratgeber', 'Kinderbuch & Jugendbuch',
       'Literatur & Unterhaltung'], dtype=object)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
array = confusion_matrix(y_pred,y_test.values)
np.fill_diagonal(array,0)
df_cm = pd.DataFrame(array, index = list(le_mapping.keys()),
                  columns = list(le_mapping.keys()))
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True,fmt=".5g")


In [ ]:
inv_map = {v: k for k, v in le_mapping.items()}

In [ ]:
y_pred_top = [inv_map[i].split(" > ")[0] for i in y_pred]

In [ ]:
y_test_top = [inv_map[i].split(" > ")[0] for i in y_test]

In [ ]:
confarray = confusion_matrix(y_test_top, y_pred_top,labels=list(set(y_test_top)))
confarray.shape

In [ ]:
conf = pd.DataFrame(confarray,columns=list(set(y_test_top)))
conf.index = list(set(y_test_top))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (10,7))
sns.heatmap(conf, annot=True,fmt="0.5g")

In [17]:
accuracy_score(y_pred_top,y_test_top)

0.8235294117647058

In [21]:
confusion_matrix(y_pred_top,y_test_top)

array([[  36,    0,    0,    1,    0,    0,    3,    1],
       [   1,  144,   11,    0,    0,    1,   35,   14],
       [   0,    8,  123,    2,    0,    9,    0,   28],
       [   0,    3,    1,  687,    2,   83,   19,    9],
       [   0,    0,    2,    2,   19,    0,    2,    8],
       [   2,    8,   19,   77,    3, 1697,   22,  123],
       [   3,   46,    4,   14,    2,    5,  363,   59],
       [   0,    2,   13,    5,    3,   35,   57,  417]])

In [2]:
import spacy
nlp = spacy.load('en')

doc = nlp(u"Apples and oranges are similar. Boots and hippos aren't.")

for token in doc:
    print(token, token.lemma, token.lemma_)

Apples 8566208034543834098 apple
and 2283656566040971221 and
oranges 2208928596161743350 orange
are 10382539506755952630 be
similar 18166476740537071113 similar
. 12646065887601541794 .
Boots 9918665227421442029 boot
and 2283656566040971221 and
hippos 6542994350242320795 hippo
are 10382539506755952630 be
n't 447765159362469301 not
. 12646065887601541794 .


In [4]:
lemmas

['duck']

### Analyse the confusions 

In [12]:
test_books = pd.read_csv("tl_test_books.csv")

In [15]:
test_books["y_test"] = y_test.values
test_books["y_pred"] = y_pred

In [23]:
mistakes = test_books[(test_books["y_pred"] == "Literatur & Unterhaltung") & (test_books["y_test"] == "Kinderbuch & Jugendbuch")]

In [24]:
len(mistakes)

94

In [ ]:
mistakes

In [51]:
pred_test = list(zip(y_pred,y_test))
e = enumerate(pred_test)
ids = [i[0] for i in e if i[1] == ("Literatur & Unterhaltung","Kinderbuch & Jugendbuch")]

In [52]:
ids

[7,
 93,
 102,
 286,
 287,
 415,
 416,
 463,
 529,
 539,
 542,
 553,
 574,
 575,
 594,
 638,
 644,
 671,
 756,
 799,
 806,
 904,
 928,
 970,
 995,
 1008,
 1066,
 1068,
 1225,
 1234,
 1291,
 1301,
 1302,
 1369,
 1435,
 1547,
 1648,
 1650,
 1684,
 1694,
 1727,
 1848,
 1849,
 1928,
 1961,
 2023,
 2034,
 2076,
 2091,
 2126,
 2180,
 2188,
 2189,
 2223,
 2334,
 2356,
 2509,
 2543,
 2544,
 2671,
 2734,
 2745,
 2781,
 2811,
 2820,
 2880,
 2963,
 2966,
 3016,
 3057,
 3070,
 3093,
 3103,
 3112,
 3127,
 3296,
 3299,
 3470,
 3536,
 3574,
 3637,
 3734,
 3805,
 3810,
 3829,
 3838,
 3910,
 3925,
 3948,
 3953,
 4092,
 4155,
 4204,
 4231]

In [36]:
pred_test[10:40]

[('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Sachbuch', 'Literatur & Unterhaltung'),
 ('Architektur & Garten', 'Architektur & Garten'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Ratgeber', 'Ratgeber'),
 ('Ganzheitliches Bewusstsein', 'Ganzheitliches Bewusstsein'),
 ('Ganzheitliches Bewusstsein', 'Ganzheitliches Bewusstsein'),
 ('Architektur & Garten', 'Architektur & Garten'),
 ('Sachbuch', 'Sachbuch'),
 ('Ratgeber', 'Ratgeber'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Kinderbuch & Jugendbuch', 'Kinderbuch & Jugendbuch'),
 ('Kinderbuch & Jugendbuch', 'Kinderbuch & Jugendbuch'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Sachbuch', 'Sachbuch'),
 ('Kinderbuch & Jugendbuch', 'Kinde